In [2]:
import json
import random
from glob import glob

In [4]:
# !wget https://huggingface.co/datasets/malaysia-ai/Malaysian-NSFW/resolve/main/malaysian-dataset-sfw-dedup.jsonl

In [9]:
labels = set()
with open('malaysian-dataset-sfw-dedup.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        labels.add(l['label'])

In [22]:
labels_ms = {
    'harassment': 'gangguan',
    'hate': 'kebencian',
    'porn': 'porno',
    'psychiatric or mental illness': 'penyakit psikiatri atau mental',
    'racist': 'sifat perkauman',
    'religion insult': 'penghinaan agama',
    'safe for work': 'selamat untuk bekerja',
    'self-harm': 'mencederakan diri sendiri',
    'sexist': 'seksis'
}

In [24]:
list(labels_ms.values())

['gangguan',
 'kebencian',
 'porno',
 'penyakit psikiatri atau mental',
 'sifat perkauman',
 'penghinaan agama',
 'selamat untuk bekerja',
 'mencederakan diri sendiri',
 'seksis']

In [25]:
labels_text = [
    '[' + ', '.join(labels) + ']',
    '{' + ', '.join(labels) + '}'
]

labels_ms_text = [
    '[' + ', '.join(list(labels_ms.values())) + ']',
    '{' + ', '.join(list(labels_ms.values())) + '}'
]

In [26]:
patterns = [
    'teks `{text}`, kelaskan teks guna labels {labels}',
    'kelaskan teks {labels}, teks `{text}`',
    'classify text {labels}, text `{text}`',
    'teks: {text}\nkelaskan text {labels}',
    'classify text {labels}\n\nteks: {text}'
]

patterns_json = [
    'teks `{text}`, kelaskan texts {labels} dan pulangkan dalam bentuk JSON {json}',
    'kelaskan teks {labels}, bagi dalam bentuk JSON {json}, teks `{text}`',
    'classify text {labels}, text `{text}`, return in JSON {json}',
    'teks: {text}\nkelaskan text {labels}, bagi dalam JSON {json}',
    'classify text {labels}\n\nteks: {text}, return JSON {json}'
]


translation = []
with open('malaysian-dataset-sfw-dedup.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        q = None
        o = None
        try:

            while True:
                c = random.randint(0, 1)
                c_l = random.randint(0, 1)
                if c_l == 0:
                    label = random.choice(labels_text)
                    o = l['label']
                else:
                    label = random.choice(labels_ms_text)
                    o = labels_ms[l['label']]

                if c == 0:
                    q = random.choice(patterns).format(text = l['text'], labels = label)
                    
                else:
                    c = random.randint(0, 1)
                    if c == 0:
                        schema =  "{'label'}"
                        d = {
                            'label': o
                        }
                    else:
                        schema =  "{'kelas'}"
                        d = {
                            'kelas': o
                        }
                        
                    
                    q = random.choice(patterns_json).format(text = l['text'], labels = label, json = schema)
                    o = json.dumps(d)

                if q and o:
                    break

            translation.append({
                'prompt_input': None,
                'input': q.strip(),
                'output': o.strip(),
            })

        except Exception as e:
            print(e)

In [27]:
len(translation)

216052

In [31]:
translation[-3]

{'prompt_input': None,
 'input': "kelaskan teks {gangguan, kebencian, porno, penyakit psikiatri atau mental, sifat perkauman, penghinaan agama, selamat untuk bekerja, mencederakan diri sendiri, seksis}, bagi dalam bentuk JSON {'label'}, teks `kebanyakan wanita yang mengikut Dajjal kerana tergoda dengan syahwat dunia dan tidak mendahulukan akalnya sebagaimana golongan lelaki.  that is so fruity - treating and thinking men are better than women`",
 'output': '{"label": "seksis"}'}

In [32]:
with open('prepared-synthetic-json-malaysian-nsfw.jsonl', 'w') as fopen:
    for t in translation:
        fopen.write(f'{json.dumps(t)}\n')

In [33]:
!ls -lh prepared-synthetic-json-malaysian-nsfw.jsonl

-rw-rw-r-- 1 husein husein 99M Jun   8 18:33 prepared-synthetic-json-malaysian-nsfw.jsonl


In [35]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='prepared-synthetic-json-malaysian-nsfw.jsonl',
    path_in_repo='prepared-synthetic-json-malaysian-nsfw.jsonl',
    repo_id='mesolitica/malaysian-nsfw-instructions',
    repo_type='dataset',
)

prepared-synthetic-json-malaysian-nsfw.jsonl:   0%|          | 0.00/104M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/malaysian-nsfw-instructions/commit/754c432ffeca32bafa0341a9a86cfc5d2b8b190d', commit_message='Upload prepared-synthetic-json-malaysian-nsfw.jsonl with huggingface_hub', commit_description='', oid='754c432ffeca32bafa0341a9a86cfc5d2b8b190d', pr_url=None, pr_revision=None, pr_num=None)